In [1]:
from training_utils import pretrain_tokenize_function
from model import ICAE
from train import ModelArguments, TrainingArguments

from peft import (
    LoraConfig,
)

model_args = ModelArguments()
training_args = TrainingArguments(output_dir="./output")

lora_config = LoraConfig(
    r=model_args.lora_r,
    lora_alpha=model_args.lora_alpha,
    lora_dropout=model_args.lora_dropout,
    bias="none",
    task_type="CAUSAL_LM"
)

2025-03-05 05:04:26.727671: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741151066.735954   28071 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741151066.739600   28071 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
model = ICAE(model_args, training_args, lora_config)
model = model.to("cuda")

You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`
The new lm_head weights will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Freezing the decoder...
trainable params: 54542336 || all params: 16115105792 || trainable%: 0.33845471884569556
Enabling gradient checkpointing...


In [1]:
import json

with open("345hop_random_true.json") as f:
    data = json.load(f)

In [2]:
from datasets import Dataset, DatasetDict

processed_data = []

for key, example in data.items():
    for in_context_key, sample in example.items():
        processed_data.append({
            "question": sample["question"],
            "query": sample["query"],
            "chain_of_thought": " ".join(sample["chain_of_thought"]),
            "answer": sample["answer"]
        })

# Convert to a Hugging Face dataset
hf_dataset = Dataset.from_list(processed_data)

In [3]:
hf_dataset[0]

{'question': 'Each lepidopteran is an insect. Each arthropod is a protostome. Every animal is multicellular. Protostomes are invertebrates. Each whale is bony. Each painted lady is a butterfly. Invertebrates are animals. Butterflies are lepidopterans. Every insect is six-legged. Every insect is an arthropod. Arthropods are not bony. Sally is a painted lady.',
 'query': 'True or false: Sally is not bony.',
 'chain_of_thought': 'Sally is a painted lady. Each painted lady is a butterfly. Sally is a butterfly. Butterflies are lepidopterans. Sally is a lepidopteran. Each lepidopteran is an insect. Sally is an insect. Every insect is an arthropod. Sally is an arthropod. Arthropods are not bony. Sally is not bony.',
 'answer': 'True'}

In [4]:
split_dataset = hf_dataset.train_test_split(test_size=0.1)
split_dataset = DatasetDict({
    "train": split_dataset["train"],
    "test": split_dataset["test"]
})

In [5]:
split_dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'query', 'chain_of_thought', 'answer'],
        num_rows: 4050
    })
    test: Dataset({
        features: ['question', 'query', 'chain_of_thought', 'answer'],
        num_rows: 450
    })
})

In [6]:
split_dataset['train']

Dataset({
    features: ['question', 'query', 'chain_of_thought', 'answer'],
    num_rows: 4050
})

In [20]:
with open("train_10_inference_examples.jsonl", "r") as f:
    train_10_inference_examples = [json.loads(line) for line in f]
    

In [23]:
train_10_inference_examples[0]

{'question': 'Tabbies are cats. Vertebrates are chordates. Bilaterians are animals. Each chordate is a bilaterian. Each cat is a feline. Each carnivore is carnivorous. Snakes are cold-blooded. Mammals are vertebrates. Animals are not unicellular. Felines are carnivores. Each mammal is not cold-blooded. Each carnivore is a mammal. Stella is a tabby.',
 'query': 'True or false: Stella is cold-blooded.',
 'chain_of_thought': 'Stella is a tabby. Tabbies are cats. Stella is a cat. Each cat is a feline. Stella is a feline. Felines are carnivores. Stella is a carnivore. Each carnivore is a mammal. Stella is a mammal. Each mammal is not cold-blooded. Stella is not cold-blooded.',
 'answer': 'False'}

In [8]:
from train import preprocess_function
import json
from datasets import Dataset, DatasetDict

with open("345hop_random_true.json") as f:
    data = json.load(f)
processed_data = []

for key, example in data.items():
    for in_context_key, sample in example.items():
        processed_data.append({
            "question": sample["question"],
            "query": sample["query"],
            "chain_of_thought": " ".join(sample["chain_of_thought"]),
            "answer": sample["answer"]
        })

hf_dataset = Dataset.from_list(processed_data)
split_dataset = hf_dataset.train_test_split(test_size=0.1)
train_dataset = split_dataset["train"]
eval_dataset = split_dataset["test"]

train_dataset = train_dataset.map(preprocess_function)
eval_dataset = eval_dataset.map(preprocess_function)

train_dataset = train_dataset.shuffle(seed=42)
eval_dataset = eval_dataset.shuffle(seed=42)

Map:   0%|          | 0/4050 [00:00<?, ? examples/s]

Map:   0%|          | 0/450 [00:00<?, ? examples/s]

In [3]:
from train import ModelArguments, TrainingArguments
from model import ICAE
from peft import (
    LoraConfig,
)
lora_config = LoraConfig(
    r=1024,
    lora_alpha=256,
    lora_dropout=0,
    bias="none",
    task_type="CAUSAL_LM"
)

model_args = ModelArguments()
model_args.model_name_or_path = "meta-llama/Llama-3.1-8B-Instruct"
training_args = TrainingArguments("./output")
model = ICAE(model_args, training_args, lora_config).to("cuda")

You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`
The new lm_head weights will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Freezing the decoder...
trainable params: 436224000 || all params: 16496787456 || trainable%: 2.6442966617803045
Enabling gradient checkpointing...


In [ ]:
train_dataset = train_dataset.map(pretrain_tokenize_function, batched=True, batch_size=1, fn_kwargs={"model": model, "mem": MEM_TOKENS, "input_type": args.input_type, "lm_ratio": training_args.lm_ratio})
eval_dataset = eval_dataset.map(pretrain_tokenize_function, batched=True, fn_kwargs={"model": model, "mem": MEM_TOKENS, "input_type": args.input_type})